In [2]:
from elasticsearch import Elasticsearch

# Adds the HTTP header 'Authorization: Basic <base64 username:password>'
es = Elasticsearch(
    "http://localhost:9200",
    basic_auth=("elastic", "lxDO2mQDVQPrxE3FRWAy")
)

print(es.ping())

False


In [3]:
import pandas as pd
df_q = pd.read_csv('C:\\Users\\A\\Music\\repos\\Information-Retrieval\\data\\questions.csv').fillna("")

In [4]:
df_q.columns

Index(['Id', 'Reputation', 'CreationDate', 'DisplayName', 'LastAccessDate',
       'WebsiteUrl', 'Location', 'AboutMe', 'Views', 'UpVotes', 'DownVotes',
       'ProfileImageUrl', 'EmailHash', 'AccountId', 'Id.1', 'PostTypeId',
       'AcceptedAnswerId', 'ParentId', 'CreationDate.1', 'DeletionDate',
       'Score', 'ViewCount', 'Body', 'OwnerUserId', 'OwnerDisplayName',
       'LastEditorUserId', 'LastEditorDisplayName', 'LastEditDate',
       'LastActivityDate', 'Title', 'Tags', 'AnswerCount', 'CommentCount',
       'FavoriteCount', 'ClosedDate', 'CommunityOwnedDate', 'ContentLicense'],
      dtype='object')

In [5]:
es.indices.create(index='train')

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'train'})

In [6]:
for i,row in df_q.iterrows():
  doc = {
      "ID" : row['Id.1'], # id.1=posts.id
      "Body" : row["Body"],
      "Tags" : (row["Tags"] if row["Tags"]!= 'non' else '')
      #"CreationDate" : row["CreationDate"], no need
      #"UpVotes" : row["UpVotes"], no need
      #"Score" : row["Score"],
      #"OwnerUserId" : row["OwnerUserId"],
  }

  es.index(index="train",id=i,document=doc)

  break
  
  

In [7]:
es.indices.get(index='train')

ObjectApiResponse({'train': {'aliases': {}, 'mappings': {'properties': {'Body': {'type': 'text', 'fields': {'keyword': {'type': 'keyword', 'ignore_above': 256}}}, 'ID': {'type': 'long'}, 'Tags': {'type': 'text', 'fields': {'keyword': {'type': 'keyword', 'ignore_above': 256}}}}}, 'settings': {'index': {'routing': {'allocation': {'include': {'_tier_preference': 'data_content'}}}, 'number_of_shards': '1', 'provided_name': 'train', 'creation_date': '1667244463693', 'number_of_replicas': '1', 'uuid': 'NeYKH-kEQOu_c_JQiiipKg', 'version': {'created': '8040399'}}}}})

In [8]:
es.indices.get_mapping(index='train')

ObjectApiResponse({'train': {'mappings': {'properties': {'Body': {'type': 'text', 'fields': {'keyword': {'type': 'keyword', 'ignore_above': 256}}}, 'ID': {'type': 'long'}, 'Tags': {'type': 'text', 'fields': {'keyword': {'type': 'keyword', 'ignore_above': 256}}}}}}})

In [9]:
es.get(index="train", id=0)

ObjectApiResponse({'_index': 'train', '_id': '0', '_version': 1, '_seq_no': 0, '_primary_term': 1, 'found': True, '_source': {'ID': 62969917, 'Body': '<p>All of a sudden, I am getting this build error in my Android project:</p>\n<pre><code>unexpected element &lt;queries&gt; found in &lt;manifest&gt;\n</code></pre>\n<p>How do I fix it?</p>\n', 'Tags': '<android><android-gradle-plugin><android-manifest><manifest>'}})

In [44]:
es.indices.delete(index="train2")

ObjectApiResponse({'acknowledged': True})

In [45]:
mapping = {
     "properties": {
            "ID": {"type": "integer"},
            "Body": {"type": "text"},
            "Tags": {"type": 'text'} #, 'analyzer': 'standard'}
            #"CreationDate": {"type": "date","format": "yyyy-MM-dd HH:mm:ss"},
            #"UpVotes": {"type": "integer"},
            #"Score": {"type": "integer"},
            #"OwnerUserId": {"type": "integer"},
         }}

es.indices.create(index="train2",mappings=mapping)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'train2'})

In [46]:
for i,row in df_q.iterrows():
  doc = {
      "ID" : row['Id.1'], # id.1=posts.id
      "Body" : row["Body"],
      "Tags" : (row["Tags"] if row["Tags"]!= 'non' else '')
      #"CreationDate" : row["CreationDate"], no need
      #"UpVotes" : row["UpVotes"], no need
      #"Score" : row["Score"],
      #"OwnerUserId" : row["OwnerUserId"],
  }
  
  es.index(index="train2",id=i,document=doc)
  break

In [47]:
es.get(index="train2", id=0)

ObjectApiResponse({'_index': 'train2', '_id': '0', '_version': 1, '_seq_no': 0, '_primary_term': 1, 'found': True, '_source': {'ID': 62969917, 'Body': '<p>All of a sudden, I am getting this build error in my Android project:</p>\n<pre><code>unexpected element &lt;queries&gt; found in &lt;manifest&gt;\n</code></pre>\n<p>How do I fix it?</p>\n', 'Tags': '<android><android-gradle-plugin><android-manifest><manifest>'}})

In [48]:
es.search(index="train", query={"match":{"Tags":"android"}})

ObjectApiResponse({'took': 1, 'timed_out': False, '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}, 'hits': {'total': {'value': 1, 'relation': 'eq'}, 'max_score': 0.45207185, 'hits': [{'_index': 'train', '_id': '0', '_score': 0.45207185, '_source': {'ID': 62969917, 'Body': '<p>All of a sudden, I am getting this build error in my Android project:</p>\n<pre><code>unexpected element &lt;queries&gt; found in &lt;manifest&gt;\n</code></pre>\n<p>How do I fix it?</p>\n', 'Tags': '<android><android-gradle-plugin><android-manifest><manifest>'}}]}})

In [22]:
es.indices.create()

AttributeError: 'IndicesClient' object has no attribute '_settings'

In [35]:
configurations = {
    "settings": {
        "analysis": {
            "analyzer": {
                "ngram_analyzer": {
                    "type": "custom",
                    "tokenizer": "standard",
                    "filter": ["lowercase"],
                },
            },
        },
    },
    "mappings":{
     "properties": {
            "ID": {"type": "integer"},
            "Body": {"type": "text"},
            "Tags": {"type": 'text', 'analyzer': 'standard'}
            #"CreationDate": {"type": "date","format": "yyyy-MM-dd HH:mm:ss"},
            #"UpVotes": {"type": "integer"},
            #"Score": {"type": "integer"},
            #"OwnerUserId": {"type": "integer"},
         }}
}

es.indices.create(index="train3",body=configurations)

<ipython-input-35-477ea0a503c0>:25: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use individual parameters.
  es.indices.create(index="train3",body=configurations)


ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'train3'})

In [36]:
for i,row in df_q.iterrows():
  doc = {
      "ID" : row['Id.1'], # id.1=posts.id
      "Body" : row["Body"],
      "Tags" : (row["Tags"] if row["Tags"]!= 'non' else '')
      #"CreationDate" : row["CreationDate"], no need
      #"UpVotes" : row["UpVotes"], no need
      #"Score" : row["Score"],
      #"OwnerUserId" : row["OwnerUserId"],
  }
  
  es.index(index="train3",id=i,document=doc)
  break

In [37]:
es.get(index="train3", id=0)

ObjectApiResponse({'_index': 'train3', '_id': '0', '_version': 1, '_seq_no': 0, '_primary_term': 1, 'found': True, '_source': {'ID': 62969917, 'Body': '<p>All of a sudden, I am getting this build error in my Android project:</p>\n<pre><code>unexpected element &lt;queries&gt; found in &lt;manifest&gt;\n</code></pre>\n<p>How do I fix it?</p>\n', 'Tags': '<android><android-gradle-plugin><android-manifest><manifest>'}})

In [40]:
es.search(index="train3", query={"match":{"Tags":"android"}})

ObjectApiResponse({'took': 4151, 'timed_out': False, '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}, 'hits': {'total': {'value': 1, 'relation': 'eq'}, 'max_score': 0.45207185, 'hits': [{'_index': 'train3', '_id': '0', '_score': 0.45207185, '_source': {'ID': 62969917, 'Body': '<p>All of a sudden, I am getting this build error in my Android project:</p>\n<pre><code>unexpected element &lt;queries&gt; found in &lt;manifest&gt;\n</code></pre>\n<p>How do I fix it?</p>\n', 'Tags': '<android><android-gradle-plugin><android-manifest><manifest>'}}]}})

In [ ]:
# Take the user's parameters and put them into a Python
# dictionary structured like an Elasticsearch query:
query_body = {"query": {"bool": {"must": {"match": {"some_field": user_request } } } }}

# call the client's search() method, and have it return results
result = es.search(index="some_index", body=query_body)

# see how many "hits" it returned using the len() function
print ("total hits:", len(result["hits"]["hits"]))


In [1]:
res = es.indices.analyze(analyzer="standard",text=["description:</strong></em> I generated a new ."])
#es.indices.analyze(body={"analyzer":"standard","text":["description:</strong></em> I generated  ."]})
res

NameError: name 'es' is not defined

In [ ]:
res['tokens']

In [ ]:
for i in res['tokens']:
    print(i['token'])


In [ ]:
res = es.indices.analyze(analyzer="stop",text=["<c++><c++17><c++20><spaceship-operator>"])
for i in res['tokens']:
    print(i['token'])

In [ ]:
IDs = []
for i in range(100):
    tags = es.get(index='questions',id=i)['_source']['Tags']
    tokenized_tags = tokenizer(tags)
    ans=1
    for tag in tokenized_tags:
        res = es.search(index='questions',query={"bool": {"must": {"match": {"Body":tag }},"filter":{"ids":{ "values": [i] }}}})
        value = res['hits']['total']['value'] # number of hits
        if value != 0:
            ans=0
            break  # no need to consider the rest of tags
    if ans: #if ans=1 means , none of tags in id's body 
        IDs.append(es.get(index='questions',id=i)['_source']['ID'])
        print(i)

In [ ]:
results = es.search(index='questions',query={"bool": { "should": [ {"term": {"Tags": "android" }},  
      ]
    }
  })


all_hits = results['hits']['hits']
print(results['hits']['total']['value'])
if results['hits']['total']['value'] == 0:
    print('NOT FOUND')  
else:
    for num, doc in enumerate(all_hits):
        print ("DOC ID:", doc["_id"], "--->", doc['_source']['Tags'], "\n")